In [1]:
import chromadb
from sentence_transformers import SentenceTransformer
from transformers import pipeline


C:\Users\belet\AppData\Roaming\Python\Python313\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:

# --- Load embedding model and ChromaDB collection (reuse from Task 2) ---
embedding_model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
client = chromadb.PersistentClient(path="./vector_store")
collection = client.get_or_create_collection(name="cfpb_complaints")


In [3]:

# --- Retriever function ---
def retrieve_relevant_chunks(question, k=5):
    question_embedding = embedding_model.encode(question).tolist()
    results = collection.query(
        query_embeddings=[question_embedding],
        n_results=k,
        include=['documents', 'metadatas']
    )
    # Flatten results for easy use
    docs = results['documents'][0]
    metas = results['metadatas'][0]
    return docs, metas


In [4]:

# --- Prompt template ---
PROMPT_TEMPLATE = (
    "You are a financial analyst assistant for CrediTrust. "
    "Your task is to answer questions about customer complaints. "
    "Use the following retrieved complaint excerpts to formulate your answer. "
    "If the context doesn't contain the answer, state that you don't have enough information.\n\n"
    "Context:\n{context}\n\nQuestion: {question}\nAnswer:"
)


In [5]:

# --- Generator (using Hugging Face pipeline, can swap for LangChain LLM) ---
# You can use 'google/flan-t5-base', 'mistralai/Mistral-7B-Instruct-v0.2', etc.
llm = pipeline("text-generation", model="google/flan-t5-base", max_new_tokens=256)


C:\Users\belet\AppData\Roaming\Python\Python313\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\belet\.cache\huggingface\hub\models--google--flan-t5-base. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Device set to use cpu
The model 'T5ForConditionalGeneration' is not supported for text-generation. Suppor

In [6]:

def generate_answer(question, context_chunks):
    context = "\n".join(context_chunks)
    prompt = PROMPT_TEMPLATE.format(context=context, question=question)
    response = llm(prompt)[0]['generated_text']
    # Remove the prompt from the output if needed
    answer = response.split("Answer:")[-1].strip()
    return answer


In [7]:

# --- Example evaluation loop ---
questions = [
    "What are the most common issues with credit card payments?",
    "How do customers feel about loan application processes?",
    "Are there complaints about mobile banking app security?",
    "What problems do users report with account closures?",
    "Is there any mention of overdraft fees in the complaints?"
]

evaluation_results = []

for q in questions:
    retrieved_chunks, retrieved_metas = retrieve_relevant_chunks(q, k=5)
    answer = generate_answer(q, retrieved_chunks)
    # For reporting, show only the first 1-2 retrieved sources
    sources = retrieved_chunks[:2]
    evaluation_results.append({
        "Question": q,
        "Generated Answer": answer,
        "Retrieved Sources": sources,
        "Quality Score": "",  # Fill in after manual review
        "Comments/Analysis": ""  # Fill in after manual review
    })


Token indices sequence length is longer than the specified maximum sequence length for this model (518 > 512). Running this sequence through the model will result in indexing errors


In [8]:

# --- Print evaluation table in Markdown format ---
print("| Question | Generated Answer | Retrieved Sources | Quality Score | Comments/Analysis |")
print("|---|---|---|---|---|")
for row in evaluation_results:
    sources_md = "<br>".join(row["Retrieved Sources"])
    print(f"| {row['Question']} | {row['Generated Answer']} | {sources_md} | {row['Quality Score']} | {row['Comments/Analysis']} |")

| Question | Generated Answer | Retrieved Sources | Quality Score | Comments/Analysis |
|---|---|---|---|---|
| What are the most common issues with credit card payments? |  | submitting complaint regarding checking account credit card navy federal credit union nfcu following issues require immediate attention overdraft fees authorized transactions nfcu repeatedly allowed transactions process checking account insufficient funds resulting overdraft fees xxxx practice appears predatory causes unnecessary financial hardship failure honor interest rate cap la xxxx xxxx activeduty service member entitled cap interest rates louisiana law xxxx xxxx despite xxxx xxxx nfcu<br>upload cards paypal use xxxxxxxx xxxxxxxx see money xxxx send occasions returned paid back minute ocasions said transaction done mistake request money back would like mention xxxx xxxx xxxx credit card name actually send money fathers mother credit cards since share name soon multiple charges parents credit cards understan